### Imports

In [43]:
from gurobipy import Model, GRB
# from collections import deque
import numpy as np
import time
import heapq

# define global variables
isMax = None # max / min sense
nodes = 0 # number of nodes
lower_bound = -np.inf # lower bound of the problem
upper_bound = np.inf # upper bound of the problems

### Model Functions 

In [44]:
# Reading problem instance from file 
def read_problem_instance(file_path):
    with open(file_path, "r") as file:
        lines = file.readlines()

    # Data Types
    # Parse data
    num_facilities = int(lines[1].split(': ')[1])
    num_customers = int(lines[2].split(': ')[1])
    # grid_size = int(lines[3].split(': ')[1].split('x')[0])

    
    facilities_coords = []
    customers_coords = []
    distances = np.zeros((num_facilities, num_customers))
    fixed_costs = np.zeros((num_facilities), dtype=int)
    facility_capacities = []
    customer_weights = []  
    cost_s = np.zeros((num_facilities, num_customers))  
    cost_l = np.zeros((num_facilities, num_customers))  

    line_idx = 5  

    # Facilities coordinates
    for _ in range(num_facilities):
        x, y = map(int, lines[line_idx].split(','))
        facilities_coords.append((x, y))
        line_idx += 1

    # Customers coordinates
    line_idx += 1
    for _ in range(num_customers):
        x, y = map(int, lines[line_idx].split(','))
        customers_coords.append((x, y))
        line_idx += 1

    # Distances
    line_idx += 1
    for i in range(num_facilities):
        for j in range(num_customers):
            distances[i][j] = float(lines[line_idx].split(': ')[1])
            line_idx += 1

    # Fixed costs for facility openings
    line_idx += 1
    for i in range(num_facilities):
        fixed_costs[i] = int(lines[line_idx].split(': ')[1])
        line_idx += 1

    # Drone operational costs (small drone)
    line_idx += 1
    for i in range(num_facilities):
        for j in range(num_customers):
            cost_s[i][j] = float(lines[line_idx].split(': ')[1])
            line_idx += 1

    # Drone operational costs (large drone)
    line_idx += 1
    for i in range(num_facilities):
        for j in range(num_customers):
            cost_l[i][j] = float(lines[line_idx].split(': ')[1])
            line_idx += 1

    # Facility capacities
    line_idx += 1
    for _ in range(num_facilities):
        facility_capacities.append(int(lines[line_idx].split(': ')[1]))
        line_idx += 1

    # Small and large drone capacities and ranges
    payload_small = int(lines[line_idx].split(': ')[1])
    payload_large = int(lines[line_idx + 1].split(': ')[1])
    range_small = int(lines[line_idx + 2].split(': ')[1])
    range_large = int(lines[line_idx + 3].split(': ')[1])

    # Check if customer weights section exists
    if line_idx + 4 < len(lines) and lines[line_idx + 4].startswith("Package weights"):
        # Read customer weights if the section exists
        line_idx += 5  # Skip to the section after the drone capacities and ranges
        for _ in range(num_customers):
            customer_weight = int(lines[line_idx].split(': ')[1])
            customer_weights.append(customer_weight)
            line_idx += 1
    else:
        # If no package weights section, assign default weights or handle error
        print("Warning: No customer weights found. Using default weights.")
        customer_weights = [1] * num_customers  # Assign default weight of 1

    Ds = int(lines[line_idx].split(': ')[1])
    Dl = int(lines[line_idx + 1].split(': ')[1])

    return {
        "num_facilities": num_facilities,
        "num_customers": num_customers,
        "distances": distances,
        "fixed_costs" : fixed_costs,
        "facility_capacities": facility_capacities,
        "payload_small": payload_small,
        "payload_large": payload_large,
        "range_small": range_small,
        "range_large": range_large,
        "customer_weights": customer_weights,
        "cost_s": cost_s,
        "cost_l": cost_l,
        "Ds" : Ds,
        "Dl" : Dl
    }

# Model initialization and solve
# Initialize the model
def model_creation(file_path):
    model = Model()
    data = read_problem_instance(file_path)

    F = range(data['num_facilities'])
    C = range(data['num_customers'])
    Ds = range(data["Ds"])
    Dl = range(data["Dl"])

    d = data['distances']
    f = data['fixed_costs']
    cs = data['cost_s']
    cl = data['cost_l']
    K = data['facility_capacities']
    W = data['customer_weights']
    Ps = data['payload_small']
    Pl = data['payload_large']
    Rs = data['range_small']
    Rl = data['range_large']
    
    ws = 1
    wl = 2

    num_vars = data['num_facilities'] + data["Ds"] * data['num_facilities'] * data['num_customers'] + data["Dl"] * data['num_facilities'] * data['num_customers']

    ub = [1] * num_vars
    lb = [0] * num_vars


    # Decision Variables 
    x = model.addVars(F, vtype=GRB.CONTINUOUS, name="x")
    ys = model.addVars(Ds, F, C, vtype=GRB.CONTINUOUS, name="ys")
    yl = model.addVars(Dl, F, C, vtype=GRB.CONTINUOUS, name="yl")

    model.update()

    # Objective Function
    penalty_large_drone = 0.5
    fixed_costs = sum(f[i] * x[i] for i in F)
    variable_costs_s  = sum( cs[i][j] * ys[s, i, j]for s in Ds for i in F for j in C )
    variable_costs_l  = sum( (cl[i][j] + penalty_large_drone) * yl[l, i, j]for l in Dl for i in F for j in C )
    variable_costs = variable_costs_s + variable_costs_l
    model.setObjective(fixed_costs + variable_costs, GRB.MINIMIZE)

    # Constraints
    for j in C:
        model.addConstr(sum(ys[s, i, j] for s in Ds for i in F) + sum(yl[l, i, j] for l in Dl for i in F) == 1)

    for i in F:
        model.addConstr(sum(ws * ys[s, i, j] for s in Ds for j in C) + sum(wl * yl[l, i, j] for l in Dl for j in C) <= K[i] * x[i])

    for s in Ds:
        for i in F:
            for j in C:
                model.addConstr(ys[s, i, j] <= x[i])

    for l in Dl:
        for i in F:
            for j in C:
                model.addConstr(yl[l, i, j] <= x[i])

    for s in Ds:
        for i in F:
            for j in C:
                model.addConstr(W[j] * ys[s, i, j] <= Ps)

    for l in Dl:
        for i in F:
            for j in C:
                model.addConstr(W[j] * yl[l, i, j] <= Pl)

    for s in Ds:
        for i in F:
            for j in C:
                model.addConstr(d[i][j] * ys[s, i, j] <= Rs)

    for l in Dl:
        for i in F:
            for j in C:
                model.addConstr(d[i][j] * yl[l, i, j] <= Rl)

    for s in Ds:
        model.addConstr(sum(ys[s, i, j] for i in F for j in C) <= 1)

    for l in Dl:
        model.addConstr(sum(yl[l, i, j] for i in F for j in C) <= 1)

    model.ModelSense = GRB.MINIMIZE 
    model.Params.method = 1  
    model.setParam('OutputFlag', 0)
    model.update()

    integer_var = num_vars

    return model, ub, lb, integer_var, num_vars, data, num_vars

### B&B and Supporting Functions

In [52]:
# A Class 'Node' That Holds the Information of a Node
class Node:
    def __init__(self, ub, lb, depth, vbasis, cbasis, branching_var, label=""):
        self.ub = ub
        self.lb = lb
        self.depth = depth
        self.vbasis = vbasis
        self.cbasis = cbasis
        self.branching_var = branching_var
        self.label = label
        self.isMax = isMax

    def __lt__(self, other):
        if self.isMax:
            return self.ub[self.branching_var] > other.ub[other.branching_var]
        else:
            return self.lb[self.branching_var] < other.lb[other.branching_var]

# Calculate If a Value is Very Close to an Integer Value
def is_nearly_integer(value, tolerance=1e-6):
    return abs(value - round(value)) <= tolerance

#### BASE BRANCHING ####
# def custom_branching_heuristic(model, data, integer_var, x_candidate):
#     for idx, is_int_var in enumerate(integer_var):
#         if is_int_var and not is_nearly_integer(x_candidate[idx]):
#             vars_have_integer_vals = False
#             selected_var_idx = idx
#             break
#         else:
#             vars_have_integer_vals = True
#             selected_var_idx = None
#     return selected_var_idx, vars_have_integer_vals

def custom_branching_heuristic(model, data, integer_var, x_candidate):
    num_facilities = data['num_facilities']
    facility_var_indices = list(range(num_facilities))  # Facility variables are indexed as the first 'num_facilities' variables.

    # Step 2: Check facility variables first
    for idx in facility_var_indices:
        if integer_var[idx] and not is_nearly_integer(x_candidate[idx]):
            # If a facility variable violates integrality, prioritize it
            return idx, False

    # Step 3: Check remaining variables
    for idx, is_int_var in enumerate(integer_var):
        if is_int_var and not is_nearly_integer(x_candidate[idx]):
            # Select the first non-facility variable violating integrality
            return idx, False

    # Step 4: If no violations found, return no variable to branch on
    return None, True

### SELECTING BRANCHING VARIABLE (STRONG BRANCHING) ###
# def select_branching_variable(model, integer_var, x_candidate, isMax, parent_obj, max_candidates=5):
#     candidates = []
#     for idx, is_int_var in enumerate(integer_var):
#         if is_int_var and not is_nearly_integer(x_candidate[idx]):
#             candidates.append((idx, abs(x_candidate[idx] - 0.5)))  # Measure fractionality
    
#     # Prioritize most fractional variables (top `max_candidates`)
#     candidates = sorted(candidates, key=lambda x: x[1], reverse=True)[:max_candidates]
#     if not candidates:
#         return -1  # No fractional variables (should not happen)
    
#     best_score = -np.inf
#     selected_var_idx = candidates[0][0]  # Default to first candidate
    
#     # Save current model state
#     original_lb = model.getAttr('LB', model.getVars())
#     original_ub = model.getAttr('UB', model.getVars())
#     original_vbasis = model.getAttr('VBasis', model.getVars())
#     original_cbasis = model.getAttr('CBasis', model.getConstrs())
    
#     for (idx, _) in candidates:
#         # Temporary left branch (x <= floor(x_val))
#         temp_ub = original_ub.copy()
#         temp_ub[idx] = np.floor(x_candidate[idx])
#         model.setAttr('UB', model.getVars(), temp_ub)
#         model.optimize()
#         obj_left = model.ObjVal if model.status == GRB.OPTIMAL else (np.inf if not isMax else -np.inf)
        
#         # Temporary right branch (x >= ceil(x_val))
#         temp_lb = original_lb.copy()
#         temp_lb[idx] = np.ceil(x_candidate[idx])
#         model.setAttr('LB', model.getVars(), temp_lb)
#         model.optimize()
#         obj_right = model.ObjVal if model.status == GRB.OPTIMAL else (np.inf if not isMax else -np.inf)
        
#         # Restore original bounds and basis
#         model.setAttr('LB', model.getVars(), original_lb)
#         model.setAttr('UB', model.getVars(), original_ub)
#         model.setAttr('VBasis', model.getVars(), original_vbasis)
#         model.setAttr('CBasis', model.getConstrs(), original_cbasis)
        
#         # Compute score using PARENT'S OBJECTIVE VALUE (parent_obj)
#         if isMax:
#             score = max(obj_left, obj_right)
#         else:
#             # Use parent_obj (original LP relaxation value) instead of model.ObjVal
#             score = (parent_obj - obj_left) * (parent_obj - obj_right)  # Product of improvements
        
#         if score > best_score:
#             best_score = score
#             selected_var_idx = idx
    
#     return selected_var_idx

# Branch & Bound Algorithm
def branch_and_bound(model, data, ub, lb, integer_var, best_bound_per_depth, nodes_per_depth, vbasis=[], cbasis=[], depth=0):
    global nodes, lower_bound, upper_bound

    # create stack using deque() structure
    # stack = deque()
    priority_queue = []

    # initialize solution list
    solutions = list()
    solutions_found = 0
    best_sol_idx = 0

    # initialize best solution
    if isMax:
        best_sol_obj = -np.inf
    else:
        best_sol_obj = np.inf

    # create root node
    root_node = Node(ub, lb, depth, vbasis, cbasis, -1, "root")
    nodes_per_depth[0] -= 1

    # solve relaxed problem
    model.optimize()

    # check if the model was solved to optimality. If not then return (infeasible).
    if model.status != GRB.OPTIMAL:
        if isMax:
            return [], -np.inf, depth
        else:
            return [], np.inf, depth

    # get the solution (variable assignments)
    x_candidate = model.getAttr('X', model.getVars())
    
    # get the objective value
    x_obj = model.ObjVal
    best_bound_per_depth[0] = x_obj

    ### SELECTING BRANCHING VARIABLE ###
    selected_var_idx, vars_have_integer_vals = custom_branching_heuristic(model, data, integer_var, x_candidate)

    # For Strong Branching 
    # vars_have_integer_vals = True
    # fractional_vars_exist = False
    # for idx, is_int_var in enumerate(integer_var):
    #     if is_int_var and not is_nearly_integer(x_candidate[idx]):
    #         vars_have_integer_vals = False
    #         fractional_vars_exist = True
    #         break

    # if not vars_have_integer_vals:
    #     selected_var_idx = select_branching_variable(model, integer_var, x_candidate, isMax, x_obj)
    #     if selected_var_idx == -1:
    #         # Fallback to first fractional variable
    #         for idx, is_int_var in enumerate(integer_var):
    #             if is_int_var and not is_nearly_integer(x_candidate[idx]):
    #                 selected_var_idx = idx
    #                 break
    
    ### SELECTING BRANCHING VARIABLE END ###

    # if we have a feasible solution in root, then terminate
    if vars_have_integer_vals:
        solutions.append([x_candidate, x_obj, depth])
        solutions_found += 1
        return solutions, best_sol_idx, solutions_found

    # otherwise update lower/upper bound for min/max respectively
    else:
        if isMax:
            upper_bound = x_obj
        else:
            lower_bound = x_obj
        
    # Retrieve vbasis and cbasis
    vbasis = model.getAttr("VBasis", model.getVars())
    cbasis = model.getAttr("CBasis", model.getConstrs())


    ### NODE CREATION ###
    # # create lower bounds and upper bounds for the variables of the child nodes
    left_lb = np.copy(lb)
    left_ub = np.copy(ub)
    right_lb = np.copy(lb)
    right_ub = np.copy(ub)

    # create left and right branches (e.g. set left: x = 0, right: x = 1 in a binary problem)
    left_ub[selected_var_idx] = np.floor(x_candidate[selected_var_idx])
    right_lb[selected_var_idx] = np.ceil(x_candidate[selected_var_idx])

    # create child nodes
    left_child = Node(left_ub, left_lb, root_node.depth + 1, vbasis.copy(), cbasis.copy(), selected_var_idx, "Left")
    right_child = Node(right_ub, right_lb, root_node.depth + 1, vbasis.copy(), cbasis.copy(), selected_var_idx, "Right")
    ### NODE CREATION END ###

    # add child nodes in stack
    # stack.append(right_child)
    # stack.append(left_child)
    # Determine priority based on problem type
    if isMax:
        priority = -x_obj  # Negative for max-heap behavior
    else:
        priority = x_obj

    heapq.heappush(priority_queue, (priority, root_node))


    # solving sub problems
    # While the stack has nodes, continue solving
    while priority_queue:
        # increment total nodes by 1
        nodes += 1

        # get the child node on top of stack
        current_priority, current_node = heapq.heappop(priority_queue)

        # remove this node from stack
        # stack.pop()

        # increase the nodes visited for current depth
        nodes_per_depth[current_node.depth] -= 1

        # warm start solver. Use the vbasis and cbasis that parent node passed to the current one.
        if (len(current_node.vbasis) != 0) and (len(current_node.cbasis) != 0):
            model.setAttr("VBasis", model.getVars(), current_node.vbasis)
            model.setAttr("CBasis", model.getConstrs(), current_node.cbasis)

        # Update the state of the model, passing the new lower bounds/upper bounds for the vars.
        # Basically, we only change the ub/lb for the branching variable. Another way is to introduce a new constraint (e.g. x_i <= ub).
        model.setAttr("LB", model.getVars(), current_node.lb)
        model.setAttr("UB", model.getVars(), current_node.ub)
        model.update()

        # optimize the model
        model.optimize()

        # Check if the model was solved to optimality. If not then do not create child nodes.
        infeasible = False
        if model.status != GRB.OPTIMAL:
            if isMax:
                infeasible = True
                x_obj = -np.inf
            else:
                infeasible = True
                x_obj = np.inf
            for i in range(current_node.depth + 1, len(nodes_per_depth)):
                nodes_per_depth[i] -= 2 * (i - current_node.depth)
        else:
            # get the solution (variable assignments)
            x_candidate = model.getAttr('X', model.getVars())

            # get the objective value
            x_obj = model.ObjVal

            # update best bound per depth if a better solution was found
            if isMax == True and x_obj > best_bound_per_depth[current_node.depth]:
                best_bound_per_depth[current_node.depth] = x_obj
            elif isMax == False and x_obj < best_bound_per_depth[current_node.depth]:
                best_bound_per_depth[current_node.depth] = x_obj

            # if we reached the final node of a depth, then update the bounds
            if nodes_per_depth[current_node.depth] == 0:
                if isMax == True:
                    upper_bound = best_bound_per_depth[current_node.depth]
                else:
                    lower_bound = best_bound_per_depth[current_node.depth]
        # if infeasible don't create children (continue searching the next node)
        if infeasible:
            continue
        
        ### SELECTING BRANCHING VARIABLE ###
        selected_var_idx, vars_have_integer_vals = custom_branching_heuristic(model, data, integer_var, x_candidate)
        
        # For Strong Branching 
        # vars_have_integer_vals = True
        # fractional_vars_exist = False
        # for idx, is_int_var in enumerate(integer_var):
        #     if is_int_var and not is_nearly_integer(x_candidate[idx]):
        #         vars_have_integer_vals = False
        #         fractional_vars_exist = True
        #         break

        # if not vars_have_integer_vals:
        #     selected_var_idx = select_branching_variable(model, integer_var, x_candidate, isMax, x_obj)
        #     if selected_var_idx == -1:
        #         # Fallback to first fractional variable
        #         for idx, is_int_var in enumerate(integer_var):
        #             if is_int_var and not is_nearly_integer(x_candidate[idx]):
        #                 selected_var_idx = idx
        #                 break
        ### SELECTING BRANCHING VARIABLE END ###

        # found feasible solution
        if vars_have_integer_vals: # integer solution
            if isMax:
                if lower_bound < x_obj: # a better solution was found
                    lower_bound = x_obj
                    if abs(lower_bound - upper_bound) < 1e-6: # optimal solution
                        # store solution, number of solutions and best sol index (and return)
                        solutions.append([x_candidate, x_obj, current_node.depth])
                        solutions_found += 1
                        if (abs(x_obj - best_sol_obj) < 1e-6) or solutions_found == 1:
                            best_sol_obj = x_obj
                            best_sol_idx = solutions_found #- 1
                        return solutions, best_sol_idx, solutions_found

                    # Not optimal. Store solution, number of solutions and best sol index (and do not expand children)
                    solutions.append([x_candidate, x_obj, current_node.depth])
                    solutions_found += 1
                    if (abs(x_obj - best_sol_obj) <= 1e-6) or solutions_found == 1:
                        best_sol_obj = x_obj
                        best_sol_idx = solutions_found #- 1

                    # remove the children nodes from each next depth
                    for i in range(current_node.depth + 1, len(nodes_per_depth)):
                        nodes_per_depth[i] -= 2 * (i - current_node.depth)
                    continue

            else:
                if upper_bound > x_obj: # better solution
                    upper_bound = x_obj # update bound
                    if abs(lower_bound - upper_bound) < 1e-6: # optimality reached
                        # store solution, number of solutions and best sol index (and return)
                        solutions.append([x_candidate, x_obj, current_node.depth])
                        solutions_found += 1
                        if (abs(x_obj - best_sol_obj) <= 1e-6) or solutions_found == 1:
                            best_sol_obj = x_obj
                            best_sol_idx = solutions_found #- 1
                        return solutions, best_sol_idx, solutions_found

                    # Not optimal. Store solution, number of solutions and best sol index (and do not expand children)
                    solutions.append([x_candidate, x_obj, current_node.depth])
                    solutions_found += 1
                    if (abs(x_obj - best_sol_obj) <= 1e-6) or solutions_found == 1:
                        best_sol_obj = x_obj
                        best_sol_idx = solutions_found #- 1

                    # remove the children nodes from each next depth
                    for i in range(current_node.depth + 1, len(nodes_per_depth)):
                        nodes_per_depth[i] -= 2 * (i - current_node.depth)
                    continue

            # do not branch further if is an equal solution
            # remove the children nodes from each next depth
            for i in range(current_node.depth + 1, len(nodes_per_depth)):
                nodes_per_depth[i] -= 2 * (i - current_node.depth)

            continue

        if isMax:
            if x_obj < lower_bound or abs(x_obj - lower_bound) < 1e-6: # cut
                # remove the children nodes from each next depth
                for i in range(current_node.depth + 1, len(nodes_per_depth)):
                    nodes_per_depth[i] -= 2 * (i - current_node.depth)
                continue
        else:
            if x_obj > upper_bound or abs(x_obj - upper_bound) < 1e-6: # cut
                # remove the children nodes from each next depth
                for i in range(current_node.depth + 1, len(nodes_per_depth)):
                    nodes_per_depth[i] -= 2 * (i - current_node.depth)
                continue

        # Retrieve vbasis and cbasis
        vbasis = model.getAttr("VBasis", model.getVars())
        cbasis = model.getAttr("CBasis", model.getConstrs())

        ### NODE CREATION ###
        # create lower bounds and upper bounds for the variables of the child nodes
        left_lb = np.copy(current_node.lb)
        left_ub = np.copy(current_node.ub)
        right_lb = np.copy(current_node.lb)
        right_ub = np.copy(current_node.ub)

        # create left and right branches (e.g. set left: x = 0, right: x = 1 in a binary problem)
        left_ub[selected_var_idx] = np.floor(x_candidate[selected_var_idx])
        right_lb[selected_var_idx] = np.ceil(x_candidate[selected_var_idx])

        # create child nodes
        left_child = Node(left_ub, left_lb, current_node.depth + 1, vbasis.copy(), cbasis.copy(), selected_var_idx,"Left")
        right_child = Node(right_ub, right_lb, current_node.depth + 1, vbasis.copy(), cbasis.copy(), selected_var_idx,"Right")
        ### NODE CREATION END ###

        # Calculate priority based on the problem type
        if isMax:
            left_priority = -left_child.ub[selected_var_idx]  # Assuming ub holds the bound
            right_priority = -right_child.ub[selected_var_idx]
        else:
            left_priority = left_child.lb[selected_var_idx]
            right_priority = right_child.lb[selected_var_idx]

        heapq.heappush(priority_queue, (left_priority, left_child))
        heapq.heappush(priority_queue, (right_priority, right_child))

    return solutions, best_sol_idx, solutions_found

In [53]:
problem_file = f"../generated_problems/class_{1}/problem_instance_{4}.txt"
model, ub, lb, integer_var, num_vars, data, num_vars = model_creation(problem_file)

# Set global parameters
nodes = 0
lower_bound = -np.inf
upper_bound = np.inf
isMax = False 
print(f"Num Vars: {num_vars}")
# Initialize structures
# Keep the best bound per depth and the total nodels visited for each depth
if isMax == True:
    best_bound_per_depth = np.array([-np.inf for i in range(num_vars)])
else:
    best_bound_per_depth = np.array([np.inf for i in range(num_vars)])

nodes_per_depth = np.zeros(num_vars + 1, dtype=float)
nodes_per_depth[0] = 1
for i in range(1, num_vars + 1):
    nodes_per_depth[i] = nodes_per_depth[i - 1] * 2
integer_var = [True] * num_vars
      
BB_start_time = time.time()
solutions, best_sol_idx, solutions_found = branch_and_bound(model= model, data= data, ub= ub, lb= lb, integer_var= integer_var, best_bound_per_depth= best_bound_per_depth,nodes_per_depth= nodes_per_depth)
BB_stop_time = time.time()
BB_elapsed = BB_stop_time - BB_start_time

all_vars = model.getVars()   
best_sol = solutions[-1][0] 

for var, val in zip(all_vars, best_sol):
    if abs(val) > 1e-12:
        print(var.VarName, "=", val)

# Print results
print("========= Optimal Solutions =========")
if solutions_found > 0:
    print("Best Solution Variables:", solutions[-1][0])
    print(f"Objective Value: {solutions[-1][1]}")
    print(f"Tree Depth: {solutions[-1][2]}")
else:
    print("Model Infeasible")

print()
# print("All Solutions:", solutions)
# print(f"Total Nodes Explored: {len(solutions)}")
print(f"BB Time: {BB_elapsed}")
# BB Time: 332.95956110954285 - No Heuristic

Set parameter Method to value 1
Num Vars: 405
x[3] = 1.0
x[4] = 1.0
ys[0,4,6] = 1.0
ys[1,4,7] = 1.0
ys[2,4,2] = 1.0
ys[3,4,1] = 1.0
ys[4,3,3] = 1.0
ys[5,3,5] = 1.0
yl[0,3,0] = 1.0
yl[3,4,4] = 1.0
========= Optimal Solutions =========
Best Solution Variables: [0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 